In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap

In [17]:
def load_pdf_data(file_path):
    # Creating a PyMuPDFLoader object with file_path
    loader = PyMuPDFLoader(file_path=file_path)
    
    # loading the PDF file
    docs = loader.load()
    
    # returning the loaded document
    return docs

In [18]:
# Responsible for splitting the documents into several chunks
def split_docs(documents, chunk_size=1024, chunk_overlap=20):
    
    # Initializing the RecursiveCharacterTextSplitter with
    # chunk_size and chunk_overlap
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    # Splitting the documents into chunks
    chunks = text_splitter.split_documents(documents=documents)
    
    # returning the document chunks
    return chunks

In [19]:
# function for loading the embedding model
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'}, # here we will run the model with CPU only
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding # keep True to compute cosine similarity
        }
    )


# Function for creating embeddings using FAISS
def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    # Creating the embeddings using FAISS
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    
    # Saving the model in current directory
    vectorstore.save_local(storing_path)
    
    # returning the vectorstore
    return vectorstore

def load_vector(embedding_model, storing_path="vectorstore"):
    vectorstore = FAISS.load_local(storing_path, embedding_model, allow_dangerous_deserialization=True)
    return vectorstore

In [20]:
prompt = """
### System:
You are an AI Assistant that follows instructions extreamly well. \
Help as much as you can.
Please say I don't know if you don't know.

### User:
{prompt}

### Response:

"""

In [21]:
template = """
### System:
You are an respectful and honest assistant. You have to answer the user's \
questions using only the context provided to you. If you don't know the answer, \
just say you don't know. Don't try to make up an answer.

### Context:
{context}

### User:
{question}

### Response:
"""

In [22]:
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, # here we are using the vectorstore as a retriever
        chain_type="stuff",
        return_source_documents=True, # including source documents in output
        chain_type_kwargs={'prompt': prompt} # customizing the prompt
    )

In [23]:
def get_response(query, chain):
    # Getting response from chain
    response = chain({'query': query})
    
    # Wrapping the text for better output in Jupyter Notebook
    wrapped_text = textwrap.fill(response['result'], width=100)
    print(wrapped_text)

In [24]:
from langchain.llms import Ollama
from langchain import PromptTemplate

In [25]:
# Loading llama3:70b from Ollama
llm = Ollama(model="llama3:70b", temperature=0)

# Loading the Embedding Model
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

In [26]:
SAVE_VEC = True
vectorstore = None
if SAVE_VEC:
    # loading and splitting the documents
    docs = load_pdf_data(file_path="data/QAOA.pdf")

    documents = split_docs(documents=docs)

    # creating vectorstore
    vectorstore = create_embeddings(documents, embed)
else:
    vectorstore = load_vector(embed)

# converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

In [27]:
# Creating the prompt from the template which we created before
prompt = PromptTemplate.from_template(template)

# Creating the chain
chain = load_qa_chain(retriever, llm, prompt)

In [28]:
get_response("Summarize the document", chain)

The document discusses the Quantum Approximate Optimization Algorithm (QAOA), a quantum gate model
algorithm that solves optimization problems by mapping the objective function to a Hamiltonian and
using quantum mechanical techniques to find the optimal solution. The algorithm has a simple and
monotonous structure and relatively good performance. The document also mentions various research
topics related to QAOA, including parameter setting, compilation, combinatorial optimization,
factoring, and deep learning.


In [29]:
get_response("What is QAOA", chain)

QAOA stands for Quantum Alternating Operator Ansatz. It's a quantum gate model algorithm that begins
with mapping an objective function to a Hamiltonian, bringing the problem into Hilbert space. Then,
it uses quantum mechanical techniques to obtain the expectation value of the Hamiltonian and
iteratively finds the parameters that optimize this expectation value.


In [30]:
get_response("What is BERT", chain)

I don't know what BERT is based on the provided context. The conversation seems to be about Quantum
Approximate Optimization Algorithm (QAOA) and its components, but there is no mention of BERT. If
you can provide more context or information about what BERT refers to, I'll do my best to help.


In [31]:
get_response("Who is SoonWook Choi", chain)

I don't know. The context provided does not mention a person named SoonWook Choi. It only mentions
J. Kim as the corresponding author of the paper.


In [32]:
resp1 = llm.invoke("What is QAOA")
print(resp1)

QAOA stands for Quantum Approximate Optimization Algorithm, (pronounced "kuh-oh-ah"). It's a quantum algorithm designed to solve complex optimization problems more efficiently than classical algorithms.

**The Problem:**
Optimization problems involve finding the best solution among many possible solutions. These problems are ubiquitous in various fields, as they can model real-world scenarios like:

* Scheduling tasks
* Allocating resources
* Portfolio optimization (finance)
* Logistics and supply chain management

Classical computers struggle to solve these problems efficiently due to their inherent complexity.

**The Solution:**
QAOA is a hybrid quantum-classical algorithm that leverages the power of quantum computing to approximate optimal solutions. It's designed to tackle complex optimization problems by exploiting the principles of quantum mechanics, such as superposition and entanglement.

Here's a high-level overview of how QAOA works:

1. **Problem Formulation**: Define the op

In [33]:
resp2 = llm.invoke("What is BERT")
print(resp2)

BERT (Bidirectional Encoder Representations from Transformers) is a pre-trained language model developed by Google in  , which has revolutionized the field of Natural Language Processing (NLP).

**What does BERT do?**

BERT is designed to understand natural language processing tasks, such as:

1. **Language Modeling**: Predicting the next word in a sentence based on the context.
2. **Text Classification**: Classifying text into categories like spam/not spam, positive/negative sentiment, etc.
3. **Named Entity Recognition**: Identifying named entities (people, organizations, locations) in text.
4. **Question Answering**: Answering questions based on the content of a passage.

**How does BERT work?**

BERT uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in a sentence. These representations are then fine-tuned for specific NLP tasks.

Here's a high-level overview of the architecture:

1. **Input Embeddings**: Word embeddings (e.g., 

In [34]:
resp3 = llm.invoke("Who is SoonWook Choi")
print(resp3)

Soon-Wook Choi is a South Korean professional golfer who has had a successful career in the sport. Here are some key facts about him:

**Early Life and Amateur Career**: Born on June  , 1963, in Seoul, South Korea, Choi started playing golf at the age of 12. He won several amateur tournaments in his home country before turning pro in 1986.

**Professional Career**: Choi has had a prolific career, winning over 20 professional tournaments worldwide, including multiple victories on the Asian Tour, Japanese Tour, and Korean Tour. His most notable win came at the 2003 Korea Open, a co-sanctioned event with the European Tour.

**International Appearances**: Choi has represented South Korea in several international team competitions, including the Presidents Cup (1996, 2000), the Dunhill Cup (1989-1991), and the World Cup (1989-1992).

**Achievements and Honors**: Choi was named the Asian Tour's Rookie of the Year in 1987 and Player of the Year in 1994. He has also been recognized for his con

In [35]:
import os
pdf = os.listdir("./data")
for _ in pdf:
    print(_)

.DS_Store
self_rag_arxiv.pdf
BERT_arxiv.pdf
crag_arxiv.pdf
RAG_arxiv.pdf
choisoonwook.pdf
QAOA.pdf


In [36]:
pid = os.getpid()
pid

25948

In [37]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 